# Standard Kinetic Modeling
## Global analysis $-$ In-built kinetic models

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
try:
    import KiMoPack.plot_func as pf
except:
    print("General installation did not work, try to import from the same folder as a workaround")
    import plot_func as pf
%matplotlib tk

## 1) Load Data

In the first step, the raw data must be passed to the ```TA``` object. Herein, it is demonstrated how to load a single measurement file. For learning how to load and average multiple scans see the ```KiMoPack_Rudppz-tutorial_ScanHandling.ipynb```

If the work with folders and filenames is cumbersome, by replacing the filename with "gui" a window pops up and a file can be selected with the mouse. ```ta=pf.TA('gui')``` This function will work well if the analysis is performed on a local computer, the selection window can sometime be hidden behind other windows.

In this example, the transient absorption data of a Ru-complex, namely [(tbbpy)$_2$Ru(dppz)]$^{2+}$, collected upon 400 nm excitation in three different solvents, namely dichloromethane (DCM), acetonitrile (ACN) and water (H$_2$O) is analysed. The tutorial is structured in such a way that only the data recorded in one solvent can be imported and analysed. To change the solvent, the solvent must be adjusted by the ```solvent``` parameter in the following cell and all subsequent steps must be repeated.

In [ ]:
solvent = 'ACN' #'DCM' or 'H2O'
filename = 'TA_Ru-dppz_400nm_'+str(solvent)                # set name of the file to fit
filepath = os.path.join(os.getcwd(), 'Data', 'Fitting-1')  # set path to file to fit

 The data is loaded by specifying the filename (string) and path of the TA-data file (string, ```path```-variable). 

In [ ]:
ta=pf.TA(filename=filename+'.SIA',    # title of the measurement file
         path=filepath)               # path to measuremnt file

Not necessary, but you can call the TA object to get some useful instructions   

In [ ]:
ta()

## 2) Standard corrections

### 2a) Background correction and filtering
- ```Filter_data```: remove artificial data points |$\Delta$OD| > 20
- ```Background```: substract background before time_zero

In [ ]:
ta.Filter_data(value=20)                # remove artificial values
ta.Background(uplimit=-0.5)             # subtract background before time zero

### 2b) Arrival time correction

Correct for different arrival times of different probe wavelength (sometimes called chirp) using ```Man_chirp``` or ```Cor_Chirp``` (checks for existing chirp data). The funtion opens an interactive 2D-contour plot of the TA data in a specified delay-time window. A desired colour map can be chosen to enable a good correction (see https://matplotlib.org/stable/tutorials/colors/colormaps.html).

- Firstly, the **intensity range** can be altered by clicking (left-click) on the scale on top of the plot. Once a suitable range is found it needs to be accepted on the lower left button.
- Secondly, user selects (left-click) a number of points along the **dispersion curve**, which are passed to a 4$^{th}$ order polynomial approximation by a middle-click. 
- Thirdly, the point that is declared as **time zero** can be selected (right-click) and confirmed (accept-button). 

In all of the selections a left click selects, a right click removes the last point and a middle click (sometime appreviated by clicking left and right together) finishes the selection. If no middle click exists, the process automatically ends after ```max_points``` (default: 40).

![Chirp](img/Cor_Chirp.gif "chirp correction")

Try ```ta.Man_Chirp(shown_window=[-2.3,1.8])``` to follow the chirp correction on your own

In [ ]:
%matplotlib qt
ta.Cor_Chirp(shown_window=[-2.3,1.8])   # choose time-window used in the active plot

note, "shown_window" is a special option chosen here because the arrival time correction needs to be performed over an extended range. Without this option -1ps to 1ps is the Default range.

## 3) Plot the standard corrected data

In this example the pre-processed data is visualized in three plots (as indicated in the titles), namely as kinetic traces (x: $\Delta{}t$, y: $\Delta$Absorbance), transient spectra (x: $\lambda_{probe}$, y: $\Delta$Absorbance) and 2D-contour plot (x: $\lambda_{probe}$, y: $\Delta{}t$, z: $\Delta$Absorbance). Several features can be sused to alter the appearance of those plots (see Documentation or type ```ta.Plot?``` in the notebook).

The parameters ```rel_time``` and ```rel_wave``` are used to pre-select interesting $\Delta{}t$ and $\lambda_{probe}$ values to show specific kinetic traces (```plotting=[1]```) or transient spectra (```plotting=[2]```)  of the dataset. 
The ```scattercut``` argument takes a probe wavelength interval that is ignored (set to zero) in the plots, to suppress the plotting of scattered excitation light. Here the scatter region was found to be between 380 and 405 nm (excitation at 400 nm). The ```time_width_percent``` variable is set to 5%, meaning that the transient spectra are shown at the given delay time plus/minus 5% of that value (e.g. 0.522 ps means 0.5 to 0.55 ps). The respective range is indicated in the legend of the transient spectra. In all plots the unfitted data is plotted as dots, interpolated with lines (Savitzky-Golay).

In [ ]:
ta.rel_time=[0.5,1.5,20,100,500] # certain delay times for TA spectra plot
ta.rel_wave=[350,440,520,600]    # certain probe wavelengths for kinetic traces plot 
ta.scattercut=[380,405]          # ignored probe wavelength region (set to zero)
ta.time_width_percent=5          # number in percent defining a delay time region plotted in the TA spectra
ta.Plot_RAW(title='Kinetic traces at selected probe wavelengths', plotting=[1])
ta.Plot_RAW(title='TA spectra at selected delay-times', plotting=[2])
ta.Plot_RAW(title='2D-Plot', plotting=[0])

### Plot Singular Value Decomposition (SVD) Results

To estimate the number of processes that contribute to the overall spectral changes singular value decomposition can be used. Herein, the SVD component strength plot reveals that *circa* three components need to be used to describe the overall TA data.

In [ ]:
ta.Plot_RAW(title='SVD', plotting=[3], savetype='pdf')  

## 4) Fitting of the Data 
### 4a) Define fitting parameters

Upon photoexcitation of [(tbbpy)$_2$Ru(dppz)]$^{2+}$ (**Ru-dppz**) at 400 nm an ensemble of metal-to-ligand charge-transfer (MLCT) states localized in both ligand spheres, *i.e.*, $^1$MLCT$_{tbbpy}$ and $^1$MLCT$_{dppz}$ is populated. Extensive photophysical studies revealed that the subsequent excited state dynamics is determined by the polarity and hydrogen bond donor ability of the surrounding solvent molecules. It was found that long-lived emissive states are populated in polar aprotic solvents. However, this emission switches *off* when the molecules interact with water. This interesting property is based on a solvent sensitive excited state equilibrium between a non-emissive and an emissive state localized on the phenazine (phz) and phenanthroline (phen) moiety of the dppz ligand. 

#### Dichloromethane

Several studies in dichloromethane solution reveal that primarily the *bright* phen-centered  state ($^3$MLCT$_{phen}$) is populated by intersystem crossing, vibrational cooling and inter ligand hooping. This *bright* state radiatively decays within *circa* 180 ns. Hence, the photoinduced dynamics in dichloromethane is herein described by two characteristic time constants, neámely ```k0``` and ```k1```. The value for ```k1``` is fixed to 180 ns as obtained from nanosecond time-resolved studies. Additionally, a parameter for time zero (```t0```) and the pump-pulse width (```resolution```) are passed to the parameter object. For each parameter it can be specified whether it is optimized (```vary=True```) or kept constant (```vary=False```) in the fit.

In [ ]:
ta.par=lmfit.Parameters()
ta.par.add('k0',value=1/2.0, min=1/10.0, max=1/0.25, vary=True)    # rate constant optimized during fit
ta.par.add('k1',value=1/180e3, vary=False)                          # rate constant optimized during fit
ta.par.add('t0',value=0.0, min=-0.1, max=0.1, vary=False)            # time-zero parameter fixed during fit
ta.par.add('resolution', value=0.12, min=0.04, max=0.20, vary=False) # pump-pulse width parameter fixed during fit

#### Water

The composition of the initially excited states is only minorly affected by the solvent. However, the phz-centered states are stabilized in aqueous environment due to a hydrogen bond interaction of the phz-nitrogen atoms with surrounding water molecules. Therefore, it was found that rapidly a phz-centered excited state is formed in water that non-radiatively decays back to the ground state within *circa* 2 ns. It a first fast process excited states with excess electron density on both, the phen and phz sphere of the dppz ligand are populated by intersystem crossing, vibrational cooling and inter ligand hopping. The dark phz-centered state is formed upon intraligand charge-transfer in a second step. Thus, also the excited state dynamics of **Ru-dppz** in H$_2$O can be described with three kinetic parameters (```k0```, ```k1``` and ```k2```), that are added to the parameters object. In contrast to the initial guesses in ACN solution, ```k3```is optimized during the fit and initially set to 2.1 ns. 

In [ ]:
ta.par=lmfit.Parameters()
ta.time_width_percent=10
ta.par.add('k0',value=1/0.5, vary=True)           # rate constant optimized during fit
ta.par.add('k1',value=1/100, vary=True)           # rate constant optimized during fit
ta.par.add('k2',value=1/1500, vary=True)          # rate constant optimized during fit
ta.par.add('t0',value=0.0, min=-0.5, max=0.5, vary=True)            # time-zero parameter fixed during fit
ta.par.add('resolution', value=0.12, min=0.04, max=0.20, vary=False) # pump-pulse width parameterfixed during fit

#### Acetonitrile

Extensive photophysical studies in acetonitrile revealed that due to the stabiization of the charge-transfer excited states, the *dark* phz-centered state is formed from the *bright* $^3$MLCT$_{phen}$ state and decays back to the ground state on the sub-ns timescale. This formation of a long-lived long-lived $^3$MLCT$_{phen}$ state is manifested, *i.e.*, in the spectral changes at 340 and 580 nm, which can be quantitatively described by two characteristic time-constants: the first one associated with  intersystem crossing, vibrational cooling and interligand hopping and a second one attributed to the non-radiative decay of a subset of $^3$MLCT states with excess electron density on the phenazine sphere of the dppz ligand ($^3$MLCT$_{phz}$), ultimately populating the long-lived $^3$MLCT$_{phen}$ state. Hence, the three kinetic parameters ```k0```, ```k1``` and ```k2``` are added to the the parameter object. The value for ```k2``` is fixed to 180 ns as obtained from nanosecond time-resolved studies.

In [ ]:
plt.close('all')
ta.par=lmfit.Parameters()
ta.par.add('k0',value=1/2, min=1/10.0, vary=True)        # rate constant optimized during fit
ta.par.add('k1',value=1/150, min=1/200.0, vary=True)     # rate constant optimized during fit
ta.par.add('infinite') # or: ta.par.add('k2', value=180e3, vary=False)# rate constant fixed during fit
ta.par.add('t0',value=0.0, min=-0.1, max=0.1, vary=True)            # time-zero parameter fixed during fit
ta.par.add('resolution', value=0.12, min=0.04, max=0.20, vary=False) # pump-pulse width parameter fixed during fit

### 4b) Kinetic Modeling (Parallel Model)

In the parallel model a number of initially photoexcited states (herein *N*=3) are taken to decay independently from each other following first order kinetics. Thus the concentration profile for each component ($C_{ac}$) is decribed by an exponential decay ($\exp\left( -k_c \cdot \Delta{t_a}\right)$). If the width of the instrument-response function is negligible, the parallel model for the TA signals reads

$~~~~~~~~~~~~~ \displaystyle \Delta{A_{ab}} = \sum_{c=1}^{N} C_{ab} \cdot S_{cb}~~~~~$ with $a \in \lbrace\Delta{}t_1,\Delta{}t_1,\dots,\Delta{}t_m\rbrace, b \in \lbrace\lambda_1,\lambda_2,\dots,\lambda_n\rbrace$

The $S_{cb}$ matrix represents the amplitudes of the exponential decays at each probe wavelength, namely the decay associated spectra. Negative contributions describe the build-up and positive contributions the decay of $\Delta{}A_{ab}$ signals.

![Schematic sketch of the parallel model](img/Fig1_parallel_model.png "Parallel Model")

In [ ]:
# model selection (independent parallel decay)
ta.mod='exponential'        
ta.timelimits=[0.3,2000]         # set maximum and minimum time for fit
#ta.ignore_time_region=[-0.2,0.3]# alternative to blind out the region around time=0 
                                 # to avoid the artifacts but allow the use of pre-zero data
ta.Fit_Global()                  # global lifetime analysis (kinetic modeling)

### 4c) Plot Fit Results

In [ ]:
plt.close('all')   # as many plots will be generated this is a good habit.
ta.intensity_range=[-70e-3,30e-3]  # We choose a slightly different intensity range to make the dynamics visible
ta.Plot_fit_output(title='2D-Plots', plotting=[4])

In [ ]:
ta.Plot_fit_output(title='TA spectra', plotting=[3])
ta.Plot_fit_output(title='kinetic traces', plotting=[2])
ta.Plot_fit_output(title='summed TA signals', plotting=[1])

In [ ]:
ta.Plot_fit_output(title='Decay Associated Spectra', plotting=[0])
ta.Plot_fit_output(title='concentration profiles', plotting=[5])

In general a single command would be used to generate all 6 plots. The approach above was chosen to give each plot a separate and distinct title.

In [ ]:
plt.close('all')
ta.Plot_fit_output()

## 5) Save Results

The whole project, including *e.g.* the fit parameters, settings for the plots and fit results can be saved by the ```Save_project``` function. Thereby a *hdf5* file is written to the specified output folder. The pre-processed data (```save_RAW=True```) as well as the fitted data and fit results (```save_Fit=True```) can be saved as ascii files to a specified folder by ```Save_data```. Thereby the following file-types are written:

**pre-processed data**
- chirp corrected 2D-matrix at all studied delay times and probe wavelengths $-$ *_chirp_corrected_raw_matrix.dat*
- 2D-matrix at probe avelengths and delay times as set by ```bordercut``` and ```timelimits``` $-$ *_matrix used as fit input.dat*
- transient spectra at selected delay times (```rel_time```) $-$ *_chirp_corrected_RAW_Spectra.dat*
- transient spectra at all studied delay times $-$ *_measured_spectra.dat*
- kinetic traces at selected probe wavelengths (```rel_wave```) $-$ *_chirp_corrected_RAW_kinetics.dat*
- kinetic traces at all studied probe wavelengths $-$ *_measured_kinetics.dat*

**fit results**
- fitted 2D-matrix at probe avelengths and delay times as set by ```bordercut``` and ```timelimits``` $-$ *_matrix calculated during fit.dat*
- difference of fitted and pre-processed 2D-matrix $-$ *_error_matrix calculated during fit.dat*

- fitted transient spectra at selected delay times (```rel_time```) $-$ *_fitted_spectra.dat*
- fitted kinetic traces at selected probe wavelengths (```rel_wave```) $-$ *_fitted_kinetics.dat*
- optimized fit paramaters $-$ *_fit_results_parameter.par*
- decay (```mod='exponential'```) or species associated spectra (```mod='consecutive'```) $-$ *_DAS-SAS.dat*

Additionally the graphs can be saved by the ```Save_Plots```function. Thereby, temporaliy the ```save_plots_to_folder```option is set to true whereby the pre-processed (*e.g.* *_RAW_SPEK.pdf*, *_RAW_SEL.pdf*) and fitted (*e.g.* *_DAC.pdf*, *_concentrations.pdf*) data are replotted.

If just the standard filenames should be used then an empty call to the saving functions is sufficient.

In [ ]:
ta.Save_project()
ta.Save_data()
ta.Save_Powerpoint()

In the example, the files are saved to the folder of the raw data in a subfolder named 'results_solvent'. Additionally some other options are chosen to select only specific files.

In [ ]:
ta.Save_project(filename=filename+'_paral',            # set save name
                path='results_'+str(solvent))          # set name of save folder

ta.Save_data(save_RAW=False,                           # do not save the pre-processed data
             save_Fit=True,                            # save the fitted data
             filename=filename+'_paral',               # set save name
             path='results_'+str(solvent))             # set name of save folder

#ta.Save_Plots(path='results_'+str(solvent),           # set name of save folder
#              title='Ru-dppz, 400nm, '+str(solvent),  # set plot titles
#              filename=filename+'_paral',             # set save name
#              savetype='pdf',                         # set save type
#              cmap=pf.cm.PiYG)                        # define colormap

ta.Save_Powerpoint(save_RAW=False,                     # do not save pre-processed data
                   save_Fit=True,                      # save pre-processed and fitted data
                   filename=filename+'_paral',         # set save name
                   path='result_summary',              # set name of save folder
                   savetype='pdf')                     # set savetype (pdf, svg or pptx)

### 4d) Kinetic Modeling (Sequential Model)

In the sequential model initially one component is excited and irreversibly decays forming a second component. Subsequently this component irreversibly decays populating a second component. this consecutive decay repeats over multiple states (in this example three states). Like in the parallel model, the TA signals are described by the product of a concentration matrix of three species and their respective spectral weights. The latter are called species associated spectra. Each j-th SAS is a combination of the j-th and the following decay associated spectrum. Thus, the final species associated spectrum is direct proportional to the final decay associated spectrum.

![Schematic sketch of the sequential model](img/Fig2_consecutive_model.png "Parallel Model")

To fit the data within own-defined models, *i.e.*, other models than, 
```python
'exponential', 'consecutive', 'full_consecutive'
``` 
see the ```KiMoPack_Rudppz-tutorial_Fitting-2.ipynb``` tutorial.

In [ ]:
# model selection (independent parallel decay)
ta.mod='consecutive'     

# global lifetime analysis (kinetic modelling)
ta.Fit_Global()

# plot results (summed kinetics)
ta.Plot_fit_output(plotting=[1])  

## 5) Save Results

In [ ]:
ta.Save_project(filename=filename+'_seq',      # set save name
                path='results_'+str(solvent))  # set name of save folder